## Installing required modules

In [1298]:
#!conda install -y tqdm

# Module Imports

In [1299]:
import tqdm.notebook as tqdm
from copy import copy
import random
from ipywidgets import *
from IPython.display import HTML
import os
import sys

# Environment setup
Please execute once to download the word list.

In [1300]:
if not os.path.exists("wordle-answers-alphabetical.txt"):
    !wget https://gist.githubusercontent.com/cfreshman/a03ef2cba789d8cf00c08f767e0fad7b/raw/5d752e5f0702da315298a6bb5a771586d6ff445c/wordle-answers-alphabetical.txt
if not os.path.exists("wordle-allowed-guesses.txt"):
    !wget https://gist.githubusercontent.com/cfreshman/cdcdf777450c5b5301e439061d29694c/raw/de1df631b45492e0974f7affe266ec36fed736eb/wordle-allowed-guesses.txt

# Loading word list

In [1301]:
answers = open("wordle-answers-alphabetical.txt", "r").read().split("\n")
guesses = open("wordle-allowed-guesses.txt", "r").read().split("\n")

# Function for scoring word

In [1302]:
MATCH_1 =  3
APPEAR_1 = 2
NEWCHR_1 = 4
MATCH_2 =  5
APPEAR_2 = 2
NEWCHR_2 = 1
def score(word, answers, matched=["-"] * 5, appeared="", discarded =""):
    word = word.lower()
    s = 0
    found = len([s for s in matched if s != "-"]) >= 4
    found = found or len([c for c in word if c in appeared]) == 5
    for i, c in enumerate(word):
        c = c.lower()
        for ans in answers:
            if ans[i] == c:
                s += MATCH_1 if not found else MATCH_2
            elif c in ans:
                s += APPEAR_1 if not found else APPEAR_2
            if c not in appeared and c not in discarded and c not in word[0:i]:
                s += NEWCHR_1 if not found else NEWCHR_2
    return s

# Function to filter constraints and sort word by score

In [1303]:
def extract(guess, matched, appeared, discarded):
    guess2 = []
    for ans in guess:
        rejected = False
        for p in range(len(matched)):
            if matched[p] != "-" and ans[p] != matched[p]:
                rejected = True
                break
        if not rejected:
            for c in appeared:
                if c not in ans:
                    rejected = True
                    break
        if not rejected:
            for c in discarded:
                if c in ans:
                    rejected = True
                    break
        if not rejected:
            guess2.append(ans)
    return guess2

def guessing(trial, guess, matched, appeared, discarded):
    guess2 = extract(guess, matched, appeared, discarded)    
    scores = [score(word, answers, matched, appeared) for word in tqdm.tqdm(guess2)]
    wordbag = [a for a in zip(guess2, scores)]
    sorted_words=sorted(wordbag, key=lambda x: -x[1])
    return sorted_words, guess2

# Functions to define game rules

In [1304]:
def check_match(word, answer):
    matched = ["-"] * 5
    appeared = ""
    discarded = ""
    for i, a, b in zip(range(5), word, answer):
        if a == b:
            matched[i] = a.lower()
    for c in word:
        if c in answer:
            appeared += c.lower()
        else:
            discarded += c.lower()
    return matched, appeared, discarded

def merge(matched1, appeared1, discarded1, matched2, appeared2, discarded2):
    matched = ["-"] * len(matched1)
    for i in range(len(matched1)):
        if matched1[i] != "-":
            matched[i] = matched1[i].lower()
        elif matched2[i] != "-":
            matched[i] = matched2[i].lower()
    appeared = set(appeared1) | set(appeared2)
    discarded = set(discarded1) | set(discarded2)
    return matched, appeared, discarded

def meet(word, matched, appeared, discarded):
    word = word.lower()
    html = ""    
    def block(color, c):
        return "<span style='display: inline-block; text-align: center; color:#ffffff; font-weight: 700; margin: 2px; width: 24px; height: 24px; background:%s'>%s</span>"%(color, c.upper())
    for i, c in enumerate(word):
        if matched[i] == c:
            html += block("#00aa00", c)
        elif c in appeared:
            html += block("#aaaa00", c)
        else:
            html += block("#888888", c)

    for i in range(len(matched)):
        if matched[i] == "-":
            return False, html
    return True, html

# Extracting words of first step.

In [1305]:
matched = ["-"]*5
appeared = ""
discarded = ""
guess2 = answers.copy()
first_words, first_guess = guessing(0, guess2, matched, appeared, discarded)
first_words = first_words

  0%|          | 0/2315 [00:00<?, ?it/s]

# Auto solver

In [1306]:
policy = "rank" # best, rank or random

## Answer detemination

In [1341]:
answer = random.choice(answers)
display(HTML("<h3 style='color: red'>word is '%s'</h3>"%answer.upper()))

In [1342]:
matched = ["-"]*5
appeared = ""
discarded = ""
#guess2 = answers.copy()
sorted_words = first_words
guess2 = first_guess.copy()

if policy not in ["best", "rank", "random"]:
    print("Bad policy '%s'"%policy)
    sys.exit(1)

for trial in range(6):
    print("Trial #%d: "%(trial+1))
    if not sorted_words:
        sorted_words, guess2 = guessing(trial, guess2, matched, appeared, discarded)
    else:
        print("Result of first guess is pre-calcurated. skipped.")

    choice = None
    
    if policy == "best":
        choice = sorted_words[0][0]
    elif policy == "random":
        choice = random.choice(sorted_words)[0]
    elif policy == "rank":
        portion = [10 / (i+1) for i in range(0, min(10, len(sorted_words)))]
        rand_max = sum(portion)
        value = random.random() * rand_max
        index = 0
        while value > 0:
            if value <= portion[index]:
                break
            value -= portion[index]
            index += 1
        choice = sorted_words[index][0]
        
    
    matched2, appeared2, discarded2 = check_match(choice, answer)

    print("candidates :", [s[0] for s in sorted_words[:10]])
    guess2.remove(choice)

    finished, html = meet(choice, matched2, appeared2, discarded2)
    display(HTML("<div>%s</div>"%html))
    if finished:
        break
    
    matched, appeared, discarded = merge(matched, appeared, discarded, matched2, appeared2, discarded2)
    sorted_words=None

Trial #1: 
Result of first guess is pre-calcurated. skipped.
candidates : ['stare', 'arose', 'raise', 'arise', 'irate', 'saner', 'snare', 'later', 'slate', 'alter']


Trial #2: 


  0%|          | 0/9 [00:00<?, ?it/s]

candidates : ['loath', 'plait', 'plant', 'giant', 'chant', 'thank', 'twang', 'await', 'adapt']


# Manual Trial with official Wordle

In [1309]:
matched = ["-"]*5
appeared = ""
discarded = ""
#guess2 = answers.copy()
sorted_words = first_words
guess2 = first_guess.copy()

## Trial loop

In [1310]:
if not sorted_words:
    sorted_words, guess2 = guessing(trial, guess2, matched, appeared, discarded)

print("Select one word and input to Wordle.", [w[0].upper() for w in sorted_words[:5]])

Select one word and input to Wordle. ['STARE', 'AROSE', 'RAISE', 'ARISE', 'IRATE']


In [1311]:
# Input result shown by Wordle.

matched2 = ["","","","",""]
appeared2 = ""
discarded2 = ""
_, html = meet(sorted_words[0][0], matched2, appeared2, discarded2)
display(HTML("<div>%s</div>"%html))

In [1312]:
guess2.remove(sorted_words[0][0])
matched, appeared, discarded = merge(matched, appeared, discarded, matched2, appeared2, discarded2)
print(sorted_words[0][0], matched, appeared, discarded)
sorted_words = None

stare ['', '', '', '', ''] set() set()


# Play by yourself!!

In [1344]:
answer = random.choice(answers)

matched = ["-"]*5
appeared = ""
discarded = ""

total_html = ""
trial = 0
history = []
input = widgets.Text(description="Guess word.", width=200)

def on_submit(word):
    global matched, appeared, discarded, answer, answers, guesses, trial, total_html,history
    if len(word) == 5:
        word = word.lower()
        if word in history:
            return
        history.append(word)
        if word not in guesses and word not in answers:
            valid = False
            word = "     "
        else:
            valid = True
            trial += 1
        matched2, appeared2, discarded2 = check_match(word, answer)
        finished, html = meet(word, matched2, appeared2, discarded2)
        if valid:
            total_html = "%s<div>%d: %s</div>"%(total_html, trial, html)
            display(HTML(total_html))
        else:
            this_html = "<div>%d: %s</div>"%(trial, html)
            display(HTML(total_html+this_html))
        matched, appeared, discarded = merge(matched, appeared, discarded, matched2, appeared2, discarded2)
        if finished:
            print("Conguraturation!!")
            input.close()
        elif trial == 6:
            print("Answer is %s"%answer.upper())
            input.close()

result = interact(on_submit, word=input)

interactive(children=(Text(value='', description='Guess word.'), Output()), _dom_classes=('widget-interact',))

In [1345]:
#Hint

print(extract(guesses.copy() + answers.copy(), matched, appeared, discarded)[:400])

['ended', 'mened', 'newed', 'nuked', 'pened', 'pwned', 'unbed', 'unked', 'kneed', 'unfed', 'unwed']
